In [2]:
import pandas as pd

In [3]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts = [{"host": "localhost", "port": 9200}],
    http_auth = ("admin", "6Wk1Cny30WOH"),
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)
client.info()

{'name': '29fc2280ba91',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'gl9GOCGSQDGvLcL0YqBAvw',
 'version': {'distribution': 'opensearch',
  'number': '2.13.0',
  'build_type': 'tar',
  'build_hash': '7ec678d1b7c87d6e779fdef94e33623e1f1e2647',
  'build_date': '2024-03-26T00:02:39.659767978Z',
  'build_snapshot': False,
  'lucene_version': '9.10.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [4]:
import psycopg2
import pandas as pd

# Параметры подключения к базе данных
dbname = 'postgres'
user = 'serikova'
password = '12345'
host = '127.0.0.1'
port = '5432'

# Подключение к базе данных
try:
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    print("Успешное подключение к базе данных PostgreSQL")

    cur = conn.cursor()

    cur.execute("SELECT * from products")
    rows = cur.fetchall()
    df = pd.DataFrame(rows)
    df.columns = [desc[0] for desc in cur.description]
    
    cur.close()
    conn.close()

except (Exception, psycopg2.Error) as error:
    print("Ошибка при подключении к PostgreSQL:", error)
finally:
    # Закрытие соединения, если оно было установлено
    if conn:
        conn.close()


Успешное подключение к базе данных PostgreSQL


In [5]:
resp = client.search(
    index="products",
    body={
        "query": {
            "bool": {
                "must": {
                    "match_phrase": {
                        "product_nanme": "апарат",
                    }
                },
            #     "filter": {"bool": {"must_not": {"match_phrase": {"director": "tim burton"}}}},
            },
        },            
    }
)
resp

{'took': 99,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [7]:
client.indices.delete(index='products')

{'acknowledged': True}

In [8]:
index_body = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "russian": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": ["lowercase", "stop", "snowball"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "product_name": {"type": "text", "analyzer": "russian"},
            "product_desc": {"type": "text", "analyzer": "russian"},
            "product_price": {"type": "keyword"},
            "product_company": {"type": "text", "analyzer": "russian"},
            "inn": {"type": "keyword"},
            "ogrn": {"type": "keyword"},
            "okpd2": {"type": "keyword"}
        }
    }
}

# Создание индекса
response = client.indices.create(index="products", body=index_body, ignore=400)  # 400 игнорирует ошибку если индекс уже существует
print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'products'}


In [9]:
import pandas as pd
for i, row in df.iterrows():
    body = {
        "product_name": row["product_name"],
        "product_desc": row["product_desc"],
        "product_price": row["product_price"],
        "product_company": row["product_company"],
        "inn": row["inn"],
        "ogrn": row["ogrn"],
        "okpd2": row["okpd2"]
    }    
    response = client.index(index="products", id=i, body=body)
print(response)

{'_index': 'products', '_id': '27538', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 27538, '_primary_term': 1}


In [8]:
import time
import pandas as pd
import zipfile
import io
import requests
from lxml import etree
from bs4 import BeautifulSoup
from io import StringIO
from selenium import webdriver
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


class FinStatements:
    balance = None
    pnl = None
    capital_change = None
    funds_movement = None

    def __init__(self, excelfile):
        excel = pd.ExcelFile(excelfile)
        self.balance = pd.read_excel(excel, 'Balance')
        self.pnl = pd.read_excel(excel, 'Financial Result')
        self.capital_change = pd.read_excel(excel, 'Capital Change')
        self.funds_movement = pd.read_excel(excel, 'Funds Movement')

    def find_in(self, string_code, year, df=None, codes=0, years={}):
        if string_code // 1000 == 1:
            df = self.balance
            years = {2019: 16, 2018: 22, 2017: 28}
            codes = 13
        elif string_code // 1000 == 2:
            df = self.pnl
            years = {2019: 20, 2018: 27}
            codes = 15
        n = df.iloc[:, codes]
        year_column = years[year]
        index = n.index[list(n).index(str(string_code))]
        value = df.iloc[index, year_column]

        negative = -1 if value[0] == '(' or value[0] == '-' else 1
        try:
            value = int("".join([x for x in value if x.isdigit()]))
        except:
            value = 0
        return negative * value


def download_excel_from_nalog(org_n):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0',
        'Content-Type': 'application/json',
    }
    url = f'https://bo.nalog.ru/organizations-card/{org_n}'
    data = {
        "params": {
            "auditReport": False,
            "balance": True,
            "capitalChange": True,
            "clarification": False,
            "targetedFundsUsing": False,
            "correctionNumber": 0,
            "financialResult": True,
            "fundsMovement": True,
            "type": "XLS",
            "period": "2023"
        }
    }
    response = requests.get(url, headers=headers, json=data)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        download_button = soup.find('button', string='Скачать таблицей или текстом')
        
        if download_button:
            print(download_button)
            zip_url = download_button.get('data-url')
            
            if zip_url:
                zip_response = requests.get(zip_url)
                
                if zip_response.status_code == 200:
                    zip_bytes = io.BytesIO(zip_response.content)
                    
                    with zipfile.ZipFile(zip_bytes, 'r') as zf:
                        excelfile = zf.read(zf.namelist()[0])
                        return excelfile
                else:
                    print(f"Failed to download zip file. Status code: {zip_response.status_code}")
            else:
                print("No URL found for downloading zip file.")
        else:
            print("Download button not found on the page.")
    else:
        print(f"Failed to fetch page. Status code: {response.status_code}")

    return None


represent = lambda number: str(round(100 * number, 4)) + '%'
profit_margin = lambda self, year: represent(self.find_in(2400, year) / self.find_in(2110, year))
ebit_margin = lambda self, year: represent(self.find_in(2300, year) / self.find_in(2110, year))
sales_margin = lambda self, year: represent(self.find_in(2200, year) / self.find_in(2110, year))
gross_margin = lambda self, year: represent(self.find_in(2100, year) / self.find_in(2110, year))
roe = lambda self, year: represent(self.find_in(2400, year) / (self.find_in(1300, year) + self.find_in(1300, year-1)))

def get_org_n_from_nalog_ru(inns, driver):
    options = FirefoxOptions()
    options.headless = True
    ac = ActionChains(driver)
    parser = etree.HTMLParser()
    results = []
    for inn in inns:
        try:
            url = f'https://bo.nalog.ru/search?query={inn}&page=1'
            driver.get(url)
            WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#search")))
            ac.click().perform()
            tree = etree.parse(StringIO(driver.page_source), parser)
            orgs = tree.xpath('//*[@id="root"]/main/div/div/div[2]/div[2]/a')
            if len(orgs) > 0:
                id_link = orgs[0].attrib['href']
                id = "".join(list(filter(str.isdigit, id_link)))
                org_name = tree.xpath('//*[@id="root"]/main/div/div/div[2]/div[2]/a/div[1]/div[1]/span')[0].text
                results.append({'name': org_name, 'inn': inn, 'id': id})
                time.sleep(3)
        except:
            pass
    return results


def process_file(org_info, driver):
    try:
        fin = FinStatements(download_excel_from_nalog(org_info['id']))
        years = [2022, 2023]
        for year in years:
            org_info.update({year: {'profit_margin': profit_margin(fin, year), 'ebit_margin': ebit_margin(fin, year),
                                    'sales_margin': sales_margin(fin, year), 'gross_margin': gross_margin(fin, year),
                                    'roe': roe(fin, year)}})
        return org_info
    except Exception as e:
        print(f"Exception in process_file: {e}")


if __name__ == "__main__":
    inn_list = ['5321029508']  # Укажите стартовые ИНН здесь
    options = FirefoxOptions()
    options.headless = True
    driver = webdriver.Firefox(options=options)
    results = []
    for inn in inn_list:
        org_n_list = get_org_n_from_nalog_ru([inn], driver)
        for org_n in org_n_list:
            print(org_n)
            results.append(process_file(org_n, driver))
    print(results)


{'name': 'ПАО "АКРОН"', 'inn': '5321029508', 'id': '2347012'}
Download button not found on the page.
Exception in process_file: Invalid file path or buffer object type: <class 'NoneType'>
[None]


In [120]:
a = pd.read_csv('na_ratings.csv')
list_inns = a['ИНН'].astype('string').str[:-2].unique().tolist()
len(list_inns)

154

In [124]:
from selenium import webdriver
import pdfplumber
import pandas as pd
import numpy as np
import time
from typing import List
from lxml import etree
from bs4 import BeautifulSoup
from io import StringIO
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

driver = webdriver.Chrome()
ac = ActionChains(driver)
parser = etree.HTMLParser()
wait = WebDriverWait(driver, 10)

list_of_df = []
for inn in list_inns:
    try:
        url = f'https://bo.nalog.ru/search?query={inn}&page=1'
        driver.get(url)
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#search")))
        ac.click().perform()
        tree = etree.parse(StringIO(driver.page_source), parser)
        orgs = tree.xpath('//*[@id="root"]/main/div/div/div[2]/div[2]/a')
        if len(orgs) > 0:
            id_link = orgs[0].attrib['href']
            id = "".join(list(filter(str.isdigit, id_link)))
            org_name = tree.xpath('//*[@id="root"]/main/div/div/div[2]/div[2]/a/div[1]/div[1]/span')[0].text
        driver.get(f'https://bo.nalog.ru/organizations-card/{id}')
        time.sleep(5)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        dfs = soup.find_all('div', class_='tabulator-table')
        list_cols = []
        list_values = []
        list_cols.append('ИНН')
        list_values.append(inn)
        for row in dfs[2].find_all('div', class_='tabulator-row'):
            list_cols.append(row.find_all('div', class_='tabulator-cell')[0].text.strip()+'2023')
            list_cols.append(row.find_all('div', class_='tabulator-cell')[0].text.strip()+'2022')
            list_values.append(row.find_all('div', class_='tabulator-cell')[2].text)
            list_values.append(row.find_all('div', class_='tabulator-cell')[3].text)

        list_cols.append('Баланс2023')
        list_cols.append('Баланс2022')
        list_values.append(dfs[0].find_all('div', class_='tabulator-row')[-1].find_all('div', class_='tabulator-cell')[2].text.strip())
        list_values.append(dfs[0].find_all('div', class_='tabulator-row')[-1].find_all('div', class_='tabulator-cell')[3].text.strip())

        data = {}
        for col, val in zip(list_cols, list_values):
            data[col] = [val]

        list_of_df.append(pd.DataFrame(data))
    except:
        pass

driver.close()

# download_archive_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "button.button_orang.button_sm")))
# download_archive_button.click()
# time.sleep(2)
# driver.switch_to.window(driver.window_handles[1])
# time.sleep(50)
# page_source = driver.page_source

In [238]:
len(list_of_df)

154

In [239]:
full_df = pd.concat(list_of_df, ignore_index=True)
len(full_df)

154

In [241]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from lxml import etree
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import time
import random
from typing import List

driver = webdriver.Chrome()
ac = ActionChains(driver)
parser = etree.HTMLParser()
wait = WebDriverWait(driver, 10)

def fetch_company_data(inn: str) -> pd.DataFrame:
    """Fetches company data from the website for a given INN."""
    try:
        url = f'https://bo.nalog.ru/search?query={inn}&page=1'
        driver.get(url)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#search")))
        ac.click().perform()
        
        tree = etree.parse(StringIO(driver.page_source), parser)
        orgs = tree.xpath('//*[@id="root"]/main/div/div/div[2]/div[2]/a')
        if not orgs:
            return pd.DataFrame()

        id_link = orgs[0].attrib['href']
        company_id = "".join(filter(str.isdigit, id_link))
        org_name = tree.xpath('//*[@id="root"]/main/div/div/div[2]/div[2]/a/div[1]/div[1]/span')[0].text
        
        driver.get(f'https://bo.nalog.ru/organizations-card/{company_id}')
        time.sleep(random.uniform(1, 5))  
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        tables = soup.find_all('div', class_='tabulator-table')
        if not tables:
            return pd.DataFrame()

        list_cols = ['ИНН']
        list_values = [inn]
        for row in tables[2].find_all('div', class_='tabulator-row'):
            year = row.find_all('div', class_='tabulator-cell')[0].text.strip()
            list_cols.extend([f'{year}2023', f'{year}2022'])
            list_values.extend([
                row.find_all('div', class_='tabulator-cell')[2].text.strip(),
                row.find_all('div', class_='tabulator-cell')[3].text.strip()
            ])
        list_cols.extend(['Баланс2023', 'Баланс2022'])
        list_values.extend([
            tables[0].find_all('div', class_='tabulator-row')[-1].find_all('div', class_='tabulator-cell')[2].text.strip(),
            tables[0].find_all('div', class_='tabulator-row')[-1].find_all('div', class_='tabulator-cell')[3].text.strip()
        ])
        data = {col: [val] for col, val in zip(list_cols, list_values)}
        return pd.DataFrame(data)

    except Exception as e:
        print(f"Error fetching data for INN {inn}: {e}")
        return pd.DataFrame()

def main(list_inns: List[str]) -> pd.DataFrame:
    """Основная функция для сбора данных для всех ИНН в списке."""
    list_of_df = []
    
    for inn in list_inns:
        df = fetch_company_data(inn)
        if not df.empty:
            list_of_df.append(df)

    driver.quit()
    return pd.concat(list_of_df, ignore_index=True)

list_inns = ['5048028420', '7707083893']  # Список ИНН
full_df = main(list_inns)
print(full_df)


          ИНН Выручка2023 Выручка2022 Себестоимость продаж2023  \
0  5048028420           -           -                        -   

  Себестоимость продаж2022 Валовая прибыль (убыток)2023  \
0                        -                            -   

  Валовая прибыль (убыток)2022 Коммерческие расходы2023  \
0                            -                        -   

  Коммерческие расходы2022 Управленческие расходы2023  ...  \
0                        -                          -  ...   

  Налог на прибыль от операций, результат которых не включается в чистую прибыль (убыток) периода2023  \
0                                                  -                                                    

  Налог на прибыль от операций, результат которых не включается в чистую прибыль (убыток) периода2022  \
0                                                  -                                                    

  Совокупный финансовый результат периода2023  \
0                                

In [242]:
full_df

,ИНН,Выручка2023,Выручка2022,Себестоимость продаж2023,Себестоимость продаж2022,Валовая прибыль (убыток)2023,Валовая прибыль (убыток)2022,Коммерческие расходы2023,Коммерческие расходы2022,Управленческие расходы2023,...,"Налог на прибыль от операций, результат которых не включается в чистую прибыль (убыток) периода2023","Налог на прибыль от операций, результат которых не включается в чистую прибыль (убыток) периода2022",Совокупный финансовый результат периода2023,Совокупный финансовый результат периода2022,Справочно Базовая прибыль (убыток) на акцию2023,Справочно Базовая прибыль (убыток) на акцию2022,Разводненная прибыль (убыток) на акцию2023,Разводненная прибыль (убыток) на акцию2022,Баланс2023,Баланс2022
0,5048028420,-,-,-,-,-,-,-,-,-,...,-,-,(1 701),-,-,-,-,-,0,0


In [154]:
import re

def clean_number_string(number_str):
    # Ищем совпадение с шаблоном (допустим пробелы и цифры)
    match = re.match(r'^(\d{1,3}(?: \d{3})*)', number_str)
    if match:
        return match.group(1)
    return number_str

def replace_bracketed_value(cell):
    match = re.search(r'\(([^)]+)\)', cell)
    if match:
        return '-' + match.group(1)
    return cell

cols_to_apply = full_df.columns.difference(['ИНН'])
full_df[cols_to_apply] = full_df[cols_to_apply].applymap(lambda x: replace_bracketed_value(str(x)))
full_df[cols_to_apply] = full_df[cols_to_apply].applymap(lambda x: clean_number_string(str(x)))

C:\Users\Sveta\AppData\Local\Temp\ipykernel_13140\2708082266.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df[cols_to_apply] = full_df[cols_to_apply].applymap(lambda x: replace_bracketed_value(str(x)))
C:\Users\Sveta\AppData\Local\Temp\ipykernel_13140\2708082266.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df[cols_to_apply] = full_df[cols_to_apply].applymap(lambda x: clean_number_string(str(x)))


In [155]:
full_df = full_df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)
full_df = full_df.replace('', np.nan)
full_df = full_df.apply(pd.to_numeric, errors='coerce').astype('Int64')


C:\Users\Sveta\AppData\Local\Temp\ipykernel_13140\2564899481.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  full_df = full_df.applymap(lambda x: x.replace(' ', '') if isinstance(x, str) else x)


In [ ]:
import time
import random
from selenium import webdriver
from bs4 import BeautifulSoup

driver = webdriver.Chrome()

for i in range(1, 51):
    url = f"https://www.ra-national.ru/ratings/?jsf=jet-engine:pk&pagenum={i}"
    driver.get(url) 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', class_='elementor-button-link elementor-button elementor-size-sm')
    download_link = links[-4].get('href')
    driver.get(download_link)
    time.sleep(random.uniform(2, 5)) 
driver.quit()

In [168]:
full_df.loc[full_df['Баланс2023'] <= 0, 'Balance_test'] = 0
full_df.loc[full_df['Баланс2023'] > 0, 'Balance_test'] = 1
full_df.loc[full_df['Валовая прибыль (убыток)2023'] <= 0, 'FinRes_test'] = 0
full_df.loc[full_df['Валовая прибыль (убыток)2023'] > 0, 'FinRes_test'] = 1
full_df.loc[full_df['Выручка2023'] * 2 < full_df['Выручка2022'], 'Revenue_test'] = 0
full_df.loc[full_df['Выручка2023'] * 2 > full_df['Выручка2022'], 'Revenue_test'] = 1

In [230]:
from bs4 import BeautifulSoup

dct = {}
for inn in list_inns:
    try:
        if 'ИНН' not in dct:
            dct['ИНН'] = []  
        dct['ИНН'].append(int(inn))  
        url = f'https://spark-interfax.ru/search?Query={inn}'
        page = requests.get(url)
        html = BeautifulSoup(page.text, "html.parser")
        company_link = 'https://spark-interfax.ru' + html.find('li', class_="search-result-list__item").find('a')['href']
        page = requests.get(company_link)
        html = BeautifulSoup(page.text, "html.parser")
        tables = html.find_all('div', class_='company-sidebar-section')

        labels = [
            ['судебные дела', 'в качестве истца', 'в качестве ответчика'],
            ['текущие производства', 'завершенные производства'],
            ['совладельцы']
        ]
        
        for table, label in zip(tables, labels):
            cells = table.find_all('a', class_="js-popup-open")
            cells = [cell for cell in cells if not cell.find(class_="js-replace-ruble")]
            for cell, name in zip(cells, label):
                num = cell.text
                if name not in dct:
                    dct[name] = []
                dct[name].append(int(num))
        
        div_element = html.find('div', class_='company-description__text').text.strip()
        match = re.search(r'(\d+)\s+тендер', div_element)
        if match:
            number = match.group(1)
        else:
            number = 0      
        if 'число тендеров' not in dct:
            dct['число тендеров'] = []
        dct['число тендеров'].append(int(number))        
    except Exception as e:
        print(f"Error processing {inn}: {e}")

Error processing <NA>: int() argument must be a string, a bytes-like object or a real number, not 'NAType'
Error processing 77012256405: 'NoneType' object has no attribute 'find'


In [232]:
dct['ИНН'].remove(77012256405)

In [233]:
keys_with_different_lengths = []

# Проходимся по ключам словаря
for key, value in dct.items():
    # Получаем длину значения, связанного с текущим ключом
    length = len(value)
    print(length)

152
152
152
152
152
152
152
152


In [235]:
interfax = pd.DataFrame(dct)
full_df = pd.merge(full_df, interfax, on='ИНН', how='left')

In [236]:
full_df

,ИНН,Выручка2023,Выручка2022,Себестоимость продаж2023,Себестоимость продаж2022,Валовая прибыль (убыток)2023,Валовая прибыль (убыток)2022,Коммерческие расходы2023,Коммерческие расходы2022,Управленческие расходы2023,...,Balance_test,FinRes_test,Revenue_test,судебные дела,в качестве истца,в качестве ответчика,текущие производства,завершенные производства,совладельцы,число тендеров
0,4025428684,328002,184658,-240280,-91697,87722,92961,<NA>,<NA>,-49616,...,1.0,1.0,1.0,7.0,1.0,3.0,0.0,59.0,2.0,0.0
1,1435126890,328002,184658,-240280,-91697,87722,92961,<NA>,<NA>,-49616,...,1.0,1.0,1.0,135.0,53.0,33.0,0.0,31.0,4.0,12.0
2,7724510200,328002,184658,-240280,-91697,87722,92961,<NA>,<NA>,-49616,...,1.0,1.0,1.0,14.0,2.0,6.0,0.0,41.0,3.0,0.0
3,3123357373,5038748,2900513,-4505509,-2495230,533239,405283,-272934,-183553,-837,...,1.0,1.0,1.0,7.0,4.0,1.0,0.0,44.0,1.0,1.0
4,7703471817,5038748,2900513,-4505509,-2495230,533239,405283,-272934,-183553,-837,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,7706810747,3168426,1453325,-125974,-121735,3042452,1331590,<NA>,<NA>,-2157050,...,1.0,1.0,1.0,8136.0,7652.0,268.0,29.0,1247.0,0.0,143.0
150,7701350084,76616171,73124201,-47202772,-33340957,29413399,39783244,-343833,-257985,-4201424,...,1.0,1.0,1.0,26.0,15.0,9.0,0.0,22.0,16.0,38.0
151,7841019595,1468629,1218828,-493725,-483212,974904,735616,0,-432,-653285,...,1.0,1.0,1.0,82.0,25.0,25.0,14.0,328.0,1.0,2.0
152,7728306068,1468629,1218828,-493725,-483212,974904,735616,0,-432,-653285,...,1.0,1.0,1.0,409.0,377.0,13.0,1.0,94.0,1.0,0.0


In [237]:
full_df.to_csv('bo_rates.csv', index=False)